In [ ]:
#filtrar tabla transfers y comprobar cerca de 70K rows

In [16]:

# Rutas
admissions = 'http://testluis.xyz/admissions.csv'
diagnoses_icd = 'http://testluis.xyz/diagnoses_icd.csv'
emar = 'http://testluis.xyz/emar.csv'
patients = 'http://testluis.xyz/patients.csv'
pharmacy = 'http://testluis.xyz/pharmacy.csv'
poe = 'http://testluis.xyz/poe.csv'
services = 'http://testluis.xyz/services.csv'
transfers = 'http://testluis.xyz/transfers.csv'

# DFs
import pandas as pd
admissions = pd.read_csv(admissions)
diagnoses_icd = pd.read_csv(diagnoses_icd)
#emar = pd.read_csv(emar)
patients = pd.read_csv(patients)
#pharmacy = pd.read_csv(pharmacy)
#poe = pd.read_csv(poe)
services = pd.read_csv(services)
transfers = pd.read_csv(transfers)

In [17]:
admissions.dtypes

subject_id               int64
hadm_id                  int64
admittime               object
dischtime               object
deathtime               object
admission_type          object
admit_provider_id       object
admission_location      object
discharge_location      object
insurance               object
language                object
marital_status          object
race                    object
edregtime               object
edouttime               object
hospital_expire_flag     int64
dtype: object

In [8]:
def eliminar_columnas_nulos(df, umbral=0.8):
    # Calcular el porcentaje de nulos en cada columna
    porcentaje_nulos = df.isnull().mean()

    # Filtrar las columnas que tienen un porcentaje de nulos mayor o igual al umbral
    columnas_a_eliminar = porcentaje_nulos[porcentaje_nulos >= umbral].index

    # Eliminar las columnas
    df_limpio = df.drop(columns=columnas_a_eliminar)

    return df_limpio

In [15]:
admissions.head(50)
#ver porcentaje de nulos > 80%
#admission dejar completa.
#cruzarla con transfers
#Mes del año
#horarios
#quitar dischtime
#revisar_expire_flag
#revisar admisiones del paciente con anterioridad
#columnas para ver admision, days_last_admission, date(admittime),
#bool de last_admm. obtener first de cada paciente admittime	- admittime	fist admission, si es el primer valos no se calcule
#Investigar como se tratan los valores que no hacen sentido como el embarzo en hombres para el modelo
#Caclculo de variables de tabla admissions

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10000032,22595853,2180-05-06 22:24:57,NaN,MED
1,10000032,22841357,2180-06-26 18:28:08,NaN,MED
2,10000032,25742920,2180-08-05 23:44:50,NaN,MED
3,10000032,29079034,2180-07-23 12:36:04,NaN,MED
4,10000068,25022803,2160-03-03 23:17:17,NaN,MED
5,10000084,23052089,2160-11-21 01:57:17,NaN,MED
6,10000084,29888819,2160-12-28 05:13:06,NaN,MED
7,10000108,27250926,2163-09-27 23:18:04,NaN,MED
8,10000117,22927623,2181-11-15 02:06:42,NaN,MED
9,10000117,27988844,2183-09-18 18:11:32,NaN,ORTHO


In [ ]:
valores_unicos = admissions['admission_location'].unique()
print(valores_unicos)

['TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM' 'WALK-IN/SELF REFERRAL'
 'PHYSICIAN REFERRAL' 'PROCEDURE SITE' 'CLINIC REFERRAL'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PACU'
 'INTERNAL TRANSFER TO OR FROM PSYCH' 'INFORMATION NOT AVAILABLE'
 'AMBULATORY SURGERY TRANSFER']


In [ ]:
patients.head(50)
#join de admissions con patients genero, anchor_age

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN
5,10000108,M,25,2163,2014 - 2016,NaN
6,10000115,M,24,2154,2017 - 2019,NaN
7,10000117,F,48,2174,2008 - 2010,NaN
8,10000178,F,59,2157,2017 - 2019,NaN
9,10000248,M,34,2192,2014 - 2016,NaN


In [ ]:
filter_transfer=transfers[transfers['careunit']==('Medical Intensive Care Unit (MICU)')]
filter_transfer


,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
14,10000032,29079034.0,39553978,admit,Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47
154,10000980,26913865.0,39765666,admit,Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27
327,10001884,26184834.0,37510196,transfer,Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30
376,10002013,21763296.0,30638341,admit,Medical Intensive Care Unit (MICU),2165-11-23 08:20:42,2165-11-23 10:25:24
429,10002155,20345487.0,32358465,admit,Medical Intensive Care Unit (MICU),2131-03-09 21:33:00,2131-03-10 18:09:21
...,...,...,...,...,...,...,...
1890778,19998878,29356037.0,30532790,transfer,Medical Intensive Care Unit (MICU),2132-08-18 14:18:47,2132-08-19 03:07:45
1890816,19999287,20175828.0,34572581,transfer,Medical Intensive Care Unit (MICU),2197-08-05 23:54:03,2197-08-07 00:04:33
1890826,19999287,22997012.0,38527782,transfer,Medical Intensive Care Unit (MICU),2197-07-26 03:52:29,2197-07-27 16:07:29
1890834,19999297,21439025.0,37364566,transfer,Medical Intensive Care Unit (MICU),2162-08-16 05:48:32,2162-08-23 06:22:41


In [ ]:
services.head(50)
#porcentaje de nulos

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10000032,22595853,2180-05-06 22:24:57,NaN,MED
1,10000032,22841357,2180-06-26 18:28:08,NaN,MED
2,10000032,25742920,2180-08-05 23:44:50,NaN,MED
3,10000032,29079034,2180-07-23 12:36:04,NaN,MED
4,10000068,25022803,2160-03-03 23:17:17,NaN,MED
5,10000084,23052089,2160-11-21 01:57:17,NaN,MED
6,10000084,29888819,2160-12-28 05:13:06,NaN,MED
7,10000108,27250926,2163-09-27 23:18:04,NaN,MED
8,10000117,22927623,2181-11-15 02:06:42,NaN,MED
9,10000117,27988844,2183-09-18 18:11:32,NaN,ORTHO


In [ ]:
diagnoses_icd.head(50)
#icd_code si es diagnostico previo o durante la estancia
#para cada paciente que diagnostico tiene
#por ahora no tomarlos
valores_unicos = diagnoses_icd['icd_code'].unique()
len(valores_unicos)

25809

In [ ]:
print(admissions.describe())

num_observaciones, num_variables = admissions.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id  hospital_expire_flag
count  4.312310e+05  4.312310e+05         431231.000000
mean   1.500766e+07  2.500314e+07              0.019964
std    2.877498e+06  2.888180e+06              0.139876
min    1.000003e+07  2.000002e+07              0.000000
25%    1.251557e+07  2.250155e+07              0.000000
50%    1.501288e+07  2.500669e+07              0.000000
75%    1.749622e+07  2.750470e+07              0.000000
max    1.999999e+07  2.999993e+07              1.000000
431231 16


In [ ]:
print(diagnoses_icd.describe())

num_observaciones, num_variables = diagnoses_icd.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id       seq_num   icd_version
count  4.756326e+06  4.756326e+06  4.756326e+06  4.756326e+06
mean   1.499966e+07  2.500011e+07  8.421183e+00  9.418274e+00
std    2.878717e+06  2.888907e+06  6.555925e+00  4.932757e-01
min    1.000003e+07  2.000002e+07  1.000000e+00  9.000000e+00
25%    1.249952e+07  2.249706e+07  3.000000e+00  9.000000e+00
50%    1.499724e+07  2.500188e+07  7.000000e+00  9.000000e+00
75%    1.749634e+07  2.750289e+07  1.200000e+01  1.000000e+01
max    1.999999e+07  2.999993e+07  3.900000e+01  1.000000e+01
4756326 5


In [ ]:
print(emar.describe())

num_observaciones, num_variables = emar.shape

print(num_observaciones, num_variables)

In [ ]:
print(patients.describe())

num_observaciones, num_variables = patients.shape

print(num_observaciones, num_variables)

         subject_id     anchor_age    anchor_year
count  2.997120e+05  299712.000000  299712.000000
mean   1.500885e+07      48.539888    2151.124056
std    2.885019e+06      20.895650      23.464598
min    1.000003e+07      18.000000    2110.000000
25%    1.250796e+07      29.000000    2131.000000
50%    1.501416e+07      48.000000    2151.000000
75%    1.750489e+07      65.000000    2171.000000
max    1.999999e+07      91.000000    2208.000000
299712 6


In [ ]:
print(pharmacy.describe())

num_observaciones, num_variables = pharmacy.shape

print(num_observaciones, num_variables)

In [ ]:
print(poe.describe())

num_observaciones, num_variables = poe.shape

print(num_observaciones, num_variables)

In [ ]:
print(services.describe())

num_observaciones, num_variables = services.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id
count  4.680290e+05  4.680290e+05
mean   1.500608e+07  2.500256e+07
std    2.877451e+06  2.888138e+06
min    1.000003e+07  2.000002e+07
25%    1.251266e+07  2.250177e+07
50%    1.500798e+07  2.500808e+07
75%    1.749302e+07  2.750253e+07
max    1.999999e+07  2.999993e+07
468029 5


In [26]:
cares=['Medical Intensive Care Unit (MICU)','Surgical Intensive Care Unit (SICU)','Medical/Surgical Intensive Care Unit (MICU/SICU)','Cardiac Vascular Intensive Care Unit (CVICU)','Neuro Surgical Intensive Care Unit (Neuro SICU)',]
transfers = transfers[transfers['careunit'].isin(cares)]

In [27]:
#transfers.head(50)
#momento de entrada a la UCI Intime
#columna calculada entre que entra la admittime	y intime
#Traslado de emergencia a planta y despues a UCI
#Medical Intensive Care Unit (MICU), Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (MICU/SICU)
valores_unicos = transfers['careunit'].unique()
print(valores_unicos)

['Medical Intensive Care Unit (MICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)'
 'Cardiac Vascular Intensive Care Unit (CVICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)']


In [28]:
print(transfers.describe())

num_observaciones, num_variables = transfers.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id   transfer_id
count  7.040300e+04  7.040300e+04  7.040300e+04
mean   1.499362e+07  2.498413e+07  3.500398e+07
std    2.881922e+06  2.880734e+06  2.888910e+06
min    1.000003e+07  2.000015e+07  3.000021e+07
25%    1.249147e+07  2.250441e+07  3.249956e+07
50%    1.499779e+07  2.496972e+07  3.500859e+07
75%    1.749902e+07  2.746318e+07  3.749563e+07
max    1.999984e+07  2.999962e+07  3.999961e+07
70403 7


In [18]:

#Eliminar filas duplicadas
admissions=eliminar_columnas_nulos(admissions,0.8)
diagnoses_icd=eliminar_columnas_nulos(diagnoses_icd,0.8)
#emar=eliminar_columnas_nulos(emar,0.8)
patients=eliminar_columnas_nulos(patients,0.8)
#pharmacy=eliminar_columnas_nulos(pharmacy,0.8)
#poe=eliminar_columnas_nulos(poe,0.8)
services=eliminar_columnas_nulos(services,0.8)
transfers=eliminar_columnas_nulos(transfers,0.8)



In [25]:
print(admissions.describe())

num_observaciones, num_variables = admissions.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id  hospital_expire_flag
count  4.312310e+05  4.312310e+05         431231.000000
mean   1.500766e+07  2.500314e+07              0.019964
std    2.877498e+06  2.888180e+06              0.139876
min    1.000003e+07  2.000002e+07              0.000000
25%    1.251557e+07  2.250155e+07              0.000000
50%    1.501288e+07  2.500669e+07              0.000000
75%    1.749622e+07  2.750470e+07              0.000000
max    1.999999e+07  2.999993e+07              1.000000
431231 15


In [24]:
admissions.head()

,subject_id,hadm_id,admittime,dischtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [23]:
print(diagnoses_icd.describe())

num_observaciones, num_variables = diagnoses_icd.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id       seq_num   icd_version
count  4.756326e+06  4.756326e+06  4.756326e+06  4.756326e+06
mean   1.499966e+07  2.500011e+07  8.421183e+00  9.418274e+00
std    2.878717e+06  2.888907e+06  6.555925e+00  4.932757e-01
min    1.000003e+07  2.000002e+07  1.000000e+00  9.000000e+00
25%    1.249952e+07  2.249706e+07  3.000000e+00  9.000000e+00
50%    1.499724e+07  2.500188e+07  7.000000e+00  9.000000e+00
75%    1.749634e+07  2.750289e+07  1.200000e+01  1.000000e+01
max    1.999999e+07  2.999993e+07  3.900000e+01  1.000000e+01
4756326 5


In [22]:
print(patients.describe())

num_observaciones, num_variables = patients.shape

print(num_observaciones, num_variables)

         subject_id     anchor_age    anchor_year
count  2.997120e+05  299712.000000  299712.000000
mean   1.500885e+07      48.539888    2151.124056
std    2.885019e+06      20.895650      23.464598
min    1.000003e+07      18.000000    2110.000000
25%    1.250796e+07      29.000000    2131.000000
50%    1.501416e+07      48.000000    2151.000000
75%    1.750489e+07      65.000000    2171.000000
max    1.999999e+07      91.000000    2208.000000
299712 5


In [21]:
print(services.describe())

num_observaciones, num_variables = services.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id
count  4.680290e+05  4.680290e+05
mean   1.500608e+07  2.500256e+07
std    2.877451e+06  2.888138e+06
min    1.000003e+07  2.000002e+07
25%    1.251266e+07  2.250177e+07
50%    1.500798e+07  2.500808e+07
75%    1.749302e+07  2.750253e+07
max    1.999999e+07  2.999993e+07
468029 4


In [20]:
print(transfers.describe())

num_observaciones, num_variables = transfers.shape

print(num_observaciones, num_variables)

         subject_id       hadm_id   transfer_id
count  1.890972e+06  1.560949e+06  1.890972e+06
mean   1.500741e+07  2.500005e+07  3.500120e+07
std    2.878974e+06  2.888445e+06  2.886887e+06
min    1.000003e+07  2.000002e+07  3.000000e+07
25%    1.251456e+07  2.249784e+07  3.250274e+07
50%    1.501187e+07  2.500050e+07  3.500176e+07
75%    1.749719e+07  2.750255e+07  3.750436e+07
max    1.999999e+07  2.999993e+07  3.999998e+07
1890972 7
